<a href="https://colab.research.google.com/github/TesisDeepcodeUcuenca/Tesis-Deepcode-Ucuenca/blob/main/Pruebas/Graficar_BER_VS_PROBABILIDADES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Graficar BER VS PROBABILIDADES Fin
import csv
import numpy as np
import torch
from scipy import stats
from tqdm import tqdm
import matplotlib.pyplot as plt
from google.colab import files

def confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = np.mean(data)
    sem = stats.sem(data)  # Error estándar de la media
    h = sem * stats.t.ppf((1 + confidence) / 2, n - 1)  # Intervalo de confianza
    return mean, mean - h, mean + h

def test(qs):
    model.eval()
    torch.manual_seed(random.randint(0, 1000))
    #print('Qs', qs)
    num_train_block = args.num_block
    errorxbit_T = torch.zeros(1, args.block_len, 1).to(device)
    promedio_errorxbit = torch.zeros(1, args.block_len, 1).to(device)
    BER_FINAL = []
    for q in qs:
        test_ber, test_bler = .0, .0
        with torch.no_grad():
            num_test_batch = int(num_train_block / (args.batch_size * test_ratio))
            for batch_idx in range(num_test_batch):
                if args.zero_padding:
                    X_test = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
                    X_test = torch.cat([X_test, torch.zeros(args.batch_size, 1, 1)], dim=1)
                    fwd_noise = torch.bernoulli(torch.full((args.batch_size, args.block_len + 1, args.code_rate), q))
                    fb_noise = torch.zeros((args.batch_size, args.block_len + 1, args.code_rate))
                else:
                    X_test = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
                    fwd_noise = torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), q))
                    fb_noise = torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), my_q_train_fb))

                X_test, fwd_noise, fb_noise = X_test.to(device), fwd_noise.to(device), fb_noise.to(device)
                X_hat_test, X_hat_test_code = model(X_test, fwd_noise, fb_noise)
                X_hat_test.to(device)
                X_hat_test_code.to(device)
                test_ber += errors_ber(X_hat_test, X_test)
                test_bler += errors_bler(X_hat_test, X_test)
                promedio_errorxbit += ber_pos(X_hat_test, X_test)

        promedio_errorxbit /= 1.0 * num_test_batch
        errorxbit_T = torch.add(errorxbit_T, promedio_errorxbit)
        test_ber /= 1.0 * num_test_batch
        test_bler /= 1.0 * num_test_batch

        print('Test Q', round(q, 3), 'with ber', float(test_ber), 'with bler', float(test_bler))
        BER_FINAL.append(q)
        BER_FINAL.append(float(test_ber))

    return errorxbit_T, BER_FINAL

#qs = [0.001, 0.003, 0.005, 0.008, 0.01, 0.05, 0.08, 0.1, 0.12, 0.15]

qs = np.linspace(0.0001, 0.15, 20)
ber_all_epochs = [[] for _ in range(len(qs))]

for epoch in tqdm(range(1, 21)):
    print(" ")
    print("Epoch: %d" % (epoch))
    print(" ")
    errorxbit_Ta, BER_FINAL = test(qs)
    for i in range(0, len(BER_FINAL), 2):
        prob_idx = int(i / 2)
        ber_all_epochs[prob_idx].append(BER_FINAL[i + 1])

mean_ber = []
ci_low = []
ci_high = []

for ber_values in ber_all_epochs:
    mean, low, high = confidence_interval(ber_values)
    mean_ber.append(mean)
    ci_low.append(low)
    ci_high.append(high)

# Almacenar los resultados en un archivo CSV
csv_filename = "ber_vs_probabilidad_error.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Probabilidad de error", "BER promedio", "Intervalo de confianza bajo", "Intervalo de confianza alto"])
    for i in range(len(qs)):
        writer.writerow([qs[i], mean_ber[i], ci_low[i], ci_high[i]])

print("Archivo CSV guardado como", csv_filename)

# Gráfico 2: BER vs Probabilidad de Error
probabilities = qs
plt.figure()
plt.plot(probabilities, mean_ber, color='b', label='Mean BER')
plt.errorbar(probabilities, mean_ber, color='r', yerr=[np.array(mean_ber) - np.array(ci_low), np.array(ci_high) - np.array(mean_ber)], fmt='.', ecolor='r', capsize=5, label='BER with 95% CI')
plt.xlabel('Probabilidad de error')
plt.ylabel('Tasa de Error de Bit (BER)')
plt.title('BER vs Probabilidad de Error')
plt.yscale('log')  # Escala logarítmica para el eje y
plt.grid(True)  # Activa el grid
plt.legend()
plt.savefig('ber_vs_probabilidad_error.png')  # Guarda la figura como un archivo PNG
plt.show()

# Descargar el archivo CSV desde Google Colab
files.download(csv_filename)

# Descargar la gráfica desde Google Colab
files.download('ber_vs_probabilidad_error.png')

print("                                                                       ")
print("                                          ███████╗    ██╗    ███╗   ██╗")
print("                                          ██╔════╝    ██║    ████╗  ██║")
print("                                          █████╗      ██║    ██╔██╗ ██║")
print("                                          ██╔══╝      ██║    ██║╚██╗██║")
print("                                          ██║         ██║    ██║ ╚████║")
print("                                          ╚═╝         ╚═╝    ╚═╝  ╚═══╝")
print("                                                                       ")
print(model)
